In [1]:
import os
from binance.spot import Spot
import time
import pandas as pd
import numpy as np
import talib

In [ ]:
os.system('wget https://github.com/TA-Lib/ta-lib/releases/download/v0.4.0/ta-lib-0.4.0-src.tar.gz')
os.system('tar -xf ta-lib-0.4.0-src.tar.gz')
os.chdir('ta-lib/')
os.system('bash configure --prefix=/usr/')
os.system('make')
os.system('sudo make install')
os.system('pip install TA-Lib')
os.chdir('/content/')

import talib

## Modello:
$ Ma = \frac{1}{n} \sum_{i = 1}^{n} x_i $

## Parametri

In [2]:
start = "2024-05-11 13:00"
end = '2024-05-13 13:00'
window1 = 5
window2 = 21

## Backtest

In [ ]:
start_obj = time.strptime(start, '%Y-%m-%d %H:%M')
start_ts = time.mktime(start_obj)
start_int = int(start_ts)
start_ts_ms = start_int * 1000

end_obj = time.strptime(end, '%Y-%m-%d %H:%M')
end_ts = time.mktime(end_obj)
end_int = int(end_ts)
end_ts_ms = end_int * 1000

spot_client = Spot(base_url = 'https://testnet.binance.vision', api_key = 'OMISSIS', api_secret = 'OMISSIS')

def dati_backtest():
    global c_df
    a = spot_client.klines('BTCUSDT', '1m', startTime = start_ts_ms, endTime = end_ts_ms)
    c_df = pd.DataFrame(a, columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'unused'])
    
    
def modello():
    global l, m, y, c_df
    c_df['x'] = talib.SMA(c_df.close, timeperiod = window1)
    c_df['y'] = talib.SMA(c_df.close, timeperiod = window2)
    c_df['x'] = c_df['x'].fillna(0)
    c_df['y'] = c_df['y'].fillna(0)
    l = l + 1
    if x == 0:
        y = x
    if x > 0:
        y = x - 1
    # se x > y e x < y a t-1 compra
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 1
            c_df['e'].loc[x] = l
    # se x > y e x > y a t-1 neutro
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0
    # se x < y e x > y a t-1 vende
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:  
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = -1
            c_df['e'].loc[x] = l
    # se x < y e x < y a t-1 neutro
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0

def backtest():
    global n, o, c_df
    n = 0
    o = 0
    if c_df['d'].loc[x] == 1:
        c_df['w'].loc[x] = c_df['close'].loc[x]
    if c_df['d'].loc[x] == -1:
        c_df['z'].loc[x] = c_df['close'].loc[x]
    c_df['w'] = c_df['w'].fillna(0)
    c_df['z'] = c_df['z'].fillna(0)
    c_df = c_df.astype({'w': 'float', 'z': 'float'})
    for p in c_df['w']:
        n += p
    for p in c_df['z']:
        o += p

dati_backtest()        

l = 0
m = 0
c_df['d'] = 0
c_df['e'] = 0
c_df['f'] = 0
c_df['h'] = 0
c_df['i'] = 0
c_df['w'] = 0
c_df['z'] = 0       

for x in range(len(c_df)):
    modello()
    backtest()
    
print('Comprato:')
print(n)
print('Venduto:')
print(o)

## Operazioni

In [3]:
spot_client = Spot(base_url = 'https://testnet.binance.vision', api_key = 'OMISSIS', api_secret = 'OMISSIS')

# da sistemare posizione dichiarazione variabili
c_df = pd.DataFrame()
l = 0
m = 0
c_df['d'] = 0
c_df['e'] = 0
c_df['f'] = 0
c_df['h'] = 0
c_df['i'] = 0
c_df['w'] = 0
c_df['z'] = 0

buy = {
    'symbol': 'BTCUSDT',
    'side': 'BUY',
    'type': 'MARKET',
    'quantity': 0.010
}

sell = {
    'symbol': 'BTCUSDT',
    'side': 'SELL',
    'type': 'MARKET',
    'quantity': 0.010
}

def dati():
    global c_df
    a = spot_client.klines('BTCUSDT', '1m', limit=1)
    b_df = pd.DataFrame(a, columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'unused'])
    c_df = pd.concat([c_df, b_df])
    c_df.reset_index(inplace=True, drop=True)
    
def modello():
    global l, m, y, c_df
    c_df['x'] = talib.SMA(c_df.close, timeperiod = window1)
    c_df['y'] = talib.SMA(c_df.close, timeperiod = window2)
    c_df['x'] = c_df['x'].fillna(0)
    c_df['y'] = c_df['y'].fillna(0)
    l = l + 1
    if x == 0:
        y = x
    if x > 0:
        y = x - 1
    # se x > y e x < y a t-1 compra
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 1
            c_df['e'].loc[x] = l
    # se x > y e x > y a t-1 neutro
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0
    # se x < y e x > y a t-1 vende
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:  
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = -1
            c_df['e'].loc[x] = l
    # se x < y e x < y a t-1 neutro
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0

def operazioni():
    if c_df['d'].loc[x] == 1:
        if c_df['e'].loc[x] == x:
            spot_client.new_order(**buy)
    if c_df['d'].loc[x] == -1:
        if c_df['e'].loc[x] == x:
            spot_client.new_order(**sell)            

for x in range(0, 60):
    time.sleep(1)
    dati()
    modello()
    operazioni()

/usr/lib/python3/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
c_df

,d,e,f,h,i,w,z,open time,open,high,...,close,volume,close time,quote asset volume,number of trades,taker buy base asset volume,taker buy quote asset volume,unused,x,y
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66085.73000000,0.01786000,1.715858e+12,1180.36197330,7.0,0.00453000,299.38968100,0,0.000,0.000000
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66085.72000000,0.01937000,1.715858e+12,1280.15141050,8.0,0.00453000,299.38968100,0,0.000,0.000000
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66078.99000000,0.03080000,1.715858e+12,2035.48950090,14.0,0.00528000,348.95407840,0,0.000,0.000000
3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66076.42000000,0.04216000,1.715858e+12,2786.14882860,18.0,0.00679000,448.74393840,0,0.000,0.000000
4,1.0,5.0,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66086.00000000,0.06102000,1.715858e+12,4032.31206500,25.0,0.00830000,548.53379840,0,66082.572,0.000000
5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66068.29000000,0.06193000,1.715858e+12,4092.43420890,26.0,0.00830000,548.53379840,0,66079.084,0.000000
6,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66066.01000000,0.06344000,1.715858e+12,4192.19388400,27.0,0.00981000,648.29347350,0,66075.142,0.000000
7,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66066.00000000,0.06495000,1.715858e+12,4291.95354400,28.0,0.00981000,648.29347350,0,66072.544,0.000000
8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66067.00000000,0.06646000,1.715858e+12,4391.71328840,30.0,0.01132000,748.05321790,0,66070.660,0.000000
9,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.715858e+12,66095.10000000,66095.10000000,...,66068.32000000,0.09781000,1.715858e+12,6462.92073460,41.0,0.04267000,2819.26066410,0,66067.124,0.000000


## Modello
$ Ema = \sum_{k = 0}^{n - 1} (1 - \alpha)^{n - k} \alpha x_k x_n$

In [ ]:
start = "2024-05-11 13:00"
end = '2024-05-13 13:00'
window1 = 5
window2 = 21

## Backtest

In [ ]:
start_obj = time.strptime(start, '%Y-%m-%d %H:%M')
start_ts = time.mktime(start_obj)
start_int = int(start_ts)
start_ts_ms = start_int * 1000

end_obj = time.strptime(end, '%Y-%m-%d %H:%M')
end_ts = time.mktime(end_obj)
end_int = int(end_ts)
end_ts_ms = end_int * 1000

spot_client = Spot(base_url = 'https://testnet.binance.vision', api_key = 'OMISSIS', api_secret = 'OMISSIS')

def dati_backtest():
    global c_df
    a = spot_client.klines('BTCUSDT', '1m', startTime = start_ts_ms, endTime = end_ts_ms)
    c_df = pd.DataFrame(a, columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'unused'])
    
    
def modello():
    global l, m, y, c_df
    c_df['x'] = talib.EMA(c_df.close, timeperiod = window1)
    c_df['y'] = talib.EMA(c_df.close, timeperiod = window2)
    c_df['x'] = c_df['x'].fillna(0)
    c_df['y'] = c_df['y'].fillna(0)
    l = l + 1
    if x == 0:
        y = x
    if x > 0:
        y = x - 1
    # se x > y e x < y a t-1 compra
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 1
            c_df['e'].loc[x] = l
    # se x > y e x > y a t-1 neutro
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0
    # se x < y e x > y a t-1 vende
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:  
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = -1
            c_df['e'].loc[x] = l
    # se x < y e x < y a t-1 neutro
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0

def backtest():
    global n, o, c_df
    n = 0
    o = 0
    if c_df['d'].loc[x] == 1:
        c_df['w'].loc[x] = c_df['close'].loc[x]
    if c_df['d'].loc[x] == -1:
        c_df['z'].loc[x] = c_df['close'].loc[x]
    c_df['w'] = c_df['w'].fillna(0)
    c_df['z'] = c_df['z'].fillna(0)
    c_df = c_df.astype({'w': 'float', 'z': 'float'})
    for p in c_df['w']:
        n += p
    for p in c_df['z']:
        o += p

dati_backtest()        

l = 0
m = 0
c_df['d'] = 0
c_df['e'] = 0
c_df['f'] = 0
c_df['h'] = 0
c_df['i'] = 0
c_df['w'] = 0
c_df['z'] = 0       

for x in range(len(c_df)):
    modello()
    backtest()
    
print('Comprato:')
print(n)
print('Venduto:')
print(o)

## Wallett

In [ ]:
print(spot_client.account())